In [1]:
import pandas as pd
import spacy
from tqdm import tqdm
from typing import List
from functools import lru_cache
from collections import Counter
import re

tqdm.pandas()

In [3]:
df = pd.read_excel('../../raw_data/CRA_train_1200.xlsx')

In [4]:
df = df.rename(columns={'Категория': 'category', 'Уровень рейтинга': 'rating'})

In [6]:
def drop_tail(text):
    if 'агентство НКР' in text:
        return text.split('Регуляторное раскрытие')[0]
    elif 'Национальное Рейтинговое Агентство' in text:
        text = text.split('(далее – НРА, Агентство)')[1]
        return text.split('ДОПОЛНИТЕЛЬНАЯ ИНФОРМАЦИЯ')[0]
    elif 'АКРА' in text:
        return text.split('Регуляторное раскрытие')[0]
    elif 'Эксперт РА' in text:
        return text.split('Контакты для СМИ')[0]
    return text

In [7]:
df['clear_text'] = df['pr_txt'].progress_apply(drop_tail)

100%|██████████| 1200/1200 [00:00<00:00, 35715.46it/s]


In [10]:
df.sample(3)

,Id,pr_txt,category,rating,clear_text
227,228,«Эксперт РА» присвоил кредитный рейтинг АО «ЮГ...,AA,AA-,«Эксперт РА» присвоил кредитный рейтинг АО «ЮГ...
592,593,Повышение кредитного рейтинга ООО «Газпромбан...,A,A-,Повышение кредитного рейтинга ООО «Газпромбан...
377,378,"Кредитный рейтинг ООО «ВУШ» (далее — Whoosh,...",A,A-,"Кредитный рейтинг ООО «ВУШ» (далее — Whoosh,..."


In [11]:
nlp = spacy.load('ru_core_news_lg')

In [13]:
@lru_cache(100000)
def clear(text):
    ner_list = ['ORG', 'LOC']
    result = []
    doc = nlp(text)
    for token in doc:
        if token.ent_type_ not in ner_list:
            result.append(token.text.lower())
    text =  ' '.join(result)
    text = text.replace('ё', 'е')
    text = text.replace('Ё', 'Е')
    text = re.sub(r'[^а-яА-Я ]', '', text)
    text = text.replace('\n', ' ')
    text = text.replace('.', '. ')
    text = text.replace(',', ', ')
    text = re.sub('\s+', ' ', text)
    return text.strip()
   

In [14]:
df['clear_text'] = df['clear_text'].progress_apply(clear)


100%|██████████| 1200/1200 [04:18<00:00,  4.65it/s]


In [15]:
df.sample(3)

,Id,pr_txt,category,rating,clear_text
115,116,"НКР подтвердило кредитный рейтинг ООО ""Патриот...",BBB,BBB-,подтвердило кредитный рейтинг на уровне прогно...
1140,1152,«Эксперт РА» понизил рейтинг компании «Группы ...,A,A,понизил рейтинг компании до уровня и изменил п...
212,213,«Эксперт РА» подтвердил кредитный рейтинг АО «...,BBB,BBB,подтвердил кредитный рейтинг на уровне сентябр...


In [16]:
@lru_cache(100000)
def lemmatize(text):
    result = []
    doc = nlp(text)
    for token in doc:
        result.append(token.lemma_)
    return ' '.join(result)

In [17]:
df['clear_text'] = df['clear_text'].progress_apply(lemmatize)


100%|██████████| 1200/1200 [03:28<00:00,  5.77it/s]


In [73]:
# words = []

In [72]:
# @lru_cache(100000)
# def get_tokens(text: str) -> List:
#     doc = nlp(str(text).lower())
#     for token in doc:
#         if token.is_stop != True and token.is_punct != True:
#             global words
#             words.append(token.text)

In [74]:
# df['clear_text'].progress_apply(get_tokens)

100%|██████████| 1200/1200 [03:50<00:00,  5.21it/s]


1       None
3       None
6       None
7       None
8       None
        ... 
873     None
900     None
968     None
1004    None
1005    None
Name: clear_text, Length: 1200, dtype: object

In [75]:
# len(words)

700665

In [76]:
# Counter(words)

Counter({'компания': 21459,
         'год': 16042,
         'уровень': 11555,
         'рейтинг': 8694,
         'агентство': 8388,
         'высокий': 7571,
         'оценка': 7049,
         'долг': 6181,
         'руб': 5572,
         'млрд': 5306,
         'риск': 5140,
         'показатель': 5023,
         'объём': 4890,
         'группа': 4646,
         'процентный': 4461,
         'платёж': 4306,
         'составить': 4276,
         'долговой': 4275,
         'рост': 4137,
         'кредитный': 4013,
         'выручка': 3982,
         'рынок': 3876,
         'низкий': 3846,
         'рентабельность': 3706,
         'чистый': 3552,
         'нагрузка': 3544,
         'период': 3375,
         'ликвидность': 3325,
         'бизнес': 3238,
         'финансовый': 3137,
         'являться': 3131,
         'денежный': 3124,
         'доля': 3122,
         'прогноз': 3028,
         'отношение': 2930,
         'снижение': 2840,
         'рейтинговый': 2798,
         'операционный': 2774,


In [18]:
nlp.Defaults.stop_words.update({'год', 'уровень', 'агентство', 'руб', 'млрд', 'компания', 'рейтинг'})

In [19]:
@lru_cache(100000)
def tokenize_it(text: str) -> List:
    result = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop != True and token.is_punct != True:
            result.append(token.text)
    return result

In [20]:
df['tokenized'] = df['clear_text'].progress_apply(tokenize_it)

100%|██████████| 1200/1200 [03:09<00:00,  6.32it/s]


In [21]:
df['tokenized_str'] = df['tokenized'].apply(lambda x: ' '.join(x))

In [22]:
df.to_csv('../../raw_data/data_prep.csv')